# **Projeto Avocado**
### **(análise de dados com Python e Pandas)**




---



### **Extração e tratamento dos dados**


In [ ]:
# bibliotecas importadas
import pandas as pd

In [ ]:
# dataset que iremos utilizar (foi preciso realizar a conversão na data)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/avocado.csv', \
                 parse_dates=['Date'], date_parser=lambda col: \
                 pd.to_datetime(col, yearfirst=True, errors='coerce'))

In [ ]:
'''
Aplicamos a conversão ao formato de data, voltando a coluna data 
para tipo String
'''
df['Date'] = df.Date.dt.strftime('%d/%m/%Y')

In [ ]:
df.head(5)

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,27/12/2015,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,20/12/2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,13/12/2015,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,06/12/2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,29/11/2015,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [ ]:
'''
Como será preciso novamente aplicar conversão à data (para que tenhamos um campo 
de data para trabalhar), criamos uma coluna de data em formato str
#''' 
df['Data padrão BR'] = df['Date']
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce', \
                            format='%d/%m/%Y')

In [ ]:
df.dtypes

Unnamed: 0                 int64
Date              datetime64[ns]
AveragePrice             float64
Total Volume             float64
4046                     float64
4225                     float64
4770                     float64
Total Bags               float64
Small Bags               float64
Large Bags               float64
XLarge Bags              float64
type                      object
year                       int64
region                    object
Data padrão BR            object
dtype: object

In [ ]:
# Os valores para gramas nos nomes das colunas são aproximados
df = df.rename(columns={"Date": "Data", "AveragePrice": "Preço médio", \
         "Total Volume": "Volume", "4046": "até 150 g", "4225": "até 300 g", \
         "4770": "até 425 g", "Total Bags": "Total de sacolas", \
         "Small Bags": "Sacolas pequenas", "Large Bags": "Sacolas grandes", \
         "XLarge Bags": "Sacolas extragrandes", "type": "Tipo", \
         "year": "Ano", "region": "Cidade/Região"})

In [ ]:
df.head(5)

,Unnamed: 0,Data,Preço médio,Volume,até 150 g,até 300 g,até 425 g,Total de sacolas,Sacolas pequenas,Sacolas grandes,Sacolas extragrandes,Tipo,Ano,Cidade/Região,Data padrão BR
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany,27/12/2015
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany,20/12/2015
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany,13/12/2015
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany,06/12/2015
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany,29/11/2015


In [ ]:
# Há uma coluna de falso index, que pode ser apagada ...
df.columns

Index(['Unnamed: 0', 'Data', 'Preço médio', 'Volume', 'até 150 g', 'até 300 g',
       'até 425 g', 'Total de sacolas', 'Sacolas pequenas', 'Sacolas grandes',
       'Sacolas extragrandes', 'Tipo', 'Ano', 'Cidade/Região',
       'Data padrão BR'],
      dtype='object')

In [ ]:
df.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [ ]:
# Descrição do dataframe
df.describe()

,Preço médio,Volume,até 150 g,até 300 g,até 425 g,Total de sacolas,Sacolas pequenas,Sacolas grandes,Sacolas extragrandes,Ano
count,18249.000000,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,18249.000000,18249.000000
mean,1.405978,8.506440e+05,2.930084e+05,2.951546e+05,2.283974e+04,2.396392e+05,1.821947e+05,5.433809e+04,3106.426507,2016.147899
std,0.402677,3.453545e+06,1.264989e+06,1.204120e+06,1.074641e+05,9.862424e+05,7.461785e+05,2.439660e+05,17692.894652,0.939938
min,0.440000,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2015.000000
25%,1.100000,1.083858e+04,8.540700e+02,3.008780e+03,0.000000e+00,5.088640e+03,2.849420e+03,1.274700e+02,0.000000,2015.000000
50%,1.370000,1.073768e+05,8.645300e+03,2.906102e+04,1.849900e+02,3.974383e+04,2.636282e+04,2.647710e+03,0.000000,2016.000000
75%,1.660000,4.329623e+05,1.110202e+05,1.502069e+05,6.243420e+03,1.107834e+05,8.333767e+04,2.202925e+04,132.500000,2017.000000
max,3.250000,6.250565e+07,2.274362e+07,2.047057e+07,2.546439e+06,1.937313e+07,1.338459e+07,5.719097e+06,551693.650000,2018.000000


In [ ]:
# Verificação de valores nulos ( não existem valores nulos no dataframe )
df.isnull().sum()

Data                    0
Preço médio             0
Volume                  0
até 150 g               0
até 300 g               0
até 425 g               0
Total de sacolas        0
Sacolas pequenas        0
Sacolas grandes         0
Sacolas extragrandes    0
Tipo                    0
Ano                     0
Cidade/Região           0
Data padrão BR          0
dtype: int64

### **Criação de novas colunas, extraindo informação com groupby**

In [ ]:
''' A coluna de receita é calculada por meio da qtd. de 
abacates vendidos X preço médio (aplicamos arredondamento de
duas casas decimais)
'''
df['Receita'] = round((df['Volume'].mul(df['Preço médio'])), 2)

In [ ]:
'''
Podemos criar colunas de porcentagens de tipo do 
abacate vendido (em relação ao volume total)
'''
df['Porcentagem 150 g'] = (df['até 150 g'] / df['Volume']) * 100
df['Porcentagem 300 g'] = (df['até 300 g'] / df['Volume']) * 100
df['Porcentagem 425 g'] = (df['até 425 g'] / df['Volume']) * 100

In [ ]:
df.sample(5)

,Data,Preço médio,Volume,até 150 g,até 300 g,até 425 g,Total de sacolas,Sacolas pequenas,Sacolas grandes,Sacolas extragrandes,Tipo,Ano,Cidade/Região,Data padrão BR,Receita,Porcentagem 150 g,Porcentagem 300 g,Porcentagem 425 g
6406,2017-01-29,1.29,168252.20,1391.78,100681.06,5482.22,60697.14,53182.31,6877.73,637.10,conventional,2017,GrandRapids,29/01/2017,217045.34,0.827199,59.839372,3.258335
13844,2016-03-27,1.33,30826.41,1420.43,16074.89,1.65,13329.44,88.94,13240.50,0.00,organic,2016,Portland,27/03/2016,40999.13,4.607835,52.146487,0.005353
13627,2016-05-29,2.07,10738.68,622.97,3962.86,5.06,6147.79,5229.58,918.21,0.00,organic,2016,Philadelphia,29/05/2016,22229.07,5.801179,36.902673,0.047119
11288,2015-05-31,1.49,10940.85,2933.55,6895.90,0.00,1111.40,1111.40,0.00,0.00,organic,2015,SanDiego,31/05/2015,16301.87,26.812816,63.028924,0.000000
5433,2016-07-03,0.93,613097.67,288161.65,142383.70,71.82,182480.50,116195.87,50307.96,15976.67,conventional,2016,Tampa,03/07/2016,570180.83,47.000937,23.223657,0.011714


In [ ]:
df['Cidade/Região'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

In [ ]:
# Group by (para todas cidades, por ano, retornando a Receita)
df.groupby(['Cidade/Região', 'Ano'])['Receita'].sum()

Cidade/Região     Ano 
Albany            2015     4764513.62
                  2016     7009219.04
                  2017     7947716.82
                  2018     2045274.34
Atlanta           2015    24515462.19
                             ...     
West              2018    91935812.47
WestTexNewMexico  2015    32242475.25
                  2016    36941988.37
                  2017    42162256.54
                  2018    10218699.01
Name: Receita, Length: 216, dtype: float64

In [ ]:
# Poderíamos observar o groupby apenas para uma cidade, por exemplo
seattle = df.loc[df['Cidade/Região'] == 'Seattle']
seattle.groupby(['Ano'])['Receita'].sum()

Ano
2015    33574695.91
2016    37881884.19
2017    43265376.61
2018    10457168.61
Name: Receita, dtype: float64

In [ ]:
# Qual o preço médio por cidade (com três casas decimais)?
round(df.groupby(['Cidade/Região'])['Preço médio'].mean(), 3)

Cidade/Região
Albany                 1.561
Atlanta                1.338
BaltimoreWashington    1.534
Boise                  1.348
Boston                 1.531
BuffaloRochester       1.517
California             1.395
Charlotte              1.606
Chicago                1.557
CincinnatiDayton       1.209
Columbus               1.253
DallasFtWorth          1.086
Denver                 1.219
Detroit                1.276
GrandRapids            1.505
GreatLakes             1.339
HarrisburgScranton     1.513
HartfordSpringfield    1.819
Houston                1.048
Indianapolis           1.314
Jacksonville           1.511
LasVegas               1.381
LosAngeles             1.216
Louisville             1.287
MiamiFtLauderdale      1.428
Midsouth               1.405
Nashville              1.212
NewOrleansMobile       1.305
NewYork                1.728
Northeast              1.602
NorthernNewEngland     1.477
Orlando                1.506
Philadelphia           1.632
PhoenixTucson          1.224


In [ ]:
# Aproveitando a pesquisa anterior, quais os menores preços médios?
media_preco = (round(df.groupby(['Cidade/Região'])['Preço médio'].mean(), 3))
media_preco.nsmallest(5)

Cidade/Região
Houston             1.048
DallasFtWorth       1.086
SouthCentral        1.101
CincinnatiDayton    1.209
Nashville           1.212
Name: Preço médio, dtype: float64

In [ ]:
# Por outro lado, as cidades com abacates mais caros são...
media_preco.nlargest(5)

Cidade/Região
HartfordSpringfield    1.819
SanFrancisco           1.804
NewYork                1.728
Philadelphia           1.632
Sacramento             1.622
Name: Preço médio, dtype: float64

### **Próxima seção!**